# Preprocessing

In [1]:
import pandas as pd
import numpy as np
#Used for pretty printing
from pprint import pprint


In [2]:
df = pd.read_csv("Data/headlinesViews.csv")
df.head()

,Unnamed: 0,title,homesection,publishdate,pageviews,popularity
0,18976,Fem danskere i Touren: Sunweb udtager Søren Kragh,Cykling,2018-07-02 17:41:01 UTC,1499,0
1,31594,Klæstrup gift i moské: Jeg er så lykkelig,Danske kendte,2018-08-13 17:40:39 UTC,227329,1
2,10062,Teenager druknet efter skub i leg: Nu er kamme...,112,2018-06-02 09:57:02 UTC,85556,1
3,23137,It-fejl i europæisk flyveplansystem er rettet,Samfund,2018-04-04 00:10:44 UTC,1346,0
4,7639,Voldtaget to timer på loftet over café: - Jeg ...,112,2018-08-06 20:42:12 UTC,303642,1


In [3]:
df.shape

(31518, 6)

In [4]:
df.popularity.value_counts()

1    15799
0    15719
Name: popularity, dtype: int64

In [5]:
df.drop(["Unnamed: 0", "publishdate","homesection","pageviews"], axis=1, inplace=True)
df.head()

,title,popularity
0,Fem danskere i Touren: Sunweb udtager Søren Kragh,0
1,Klæstrup gift i moské: Jeg er så lykkelig,1
2,Teenager druknet efter skub i leg: Nu er kamme...,1
3,It-fejl i europæisk flyveplansystem er rettet,0
4,Voldtaget to timer på loftet over café: - Jeg ...,1


In [6]:
# Make everything lowercase
df["title"] = df["title"].str.lower()
df

,title,popularity
0,fem danskere i touren: sunweb udtager søren kragh,0
1,klæstrup gift i moské: jeg er så lykkelig,1
2,teenager druknet efter skub i leg: nu er kamme...,1
3,it-fejl i europæisk flyveplansystem er rettet,0
4,voldtaget to timer på loftet over café: - jeg ...,1
...,...,...
31513,helt til hest: politiet mangler borgernes hjæl...,0
31514,kæmpe ballade under pokalfinale: politiet brug...,1
31515,lavede sexbånd med kardashian: jeg er videre,0
31516,politiet bekræfter: nu skal ronaldo afhøres,1


In [7]:
# Remove all punctuation
df['title'] = df['title'].str.replace('[^\w\s]','')
df['title'].head()

0     fem danskere i touren sunweb udtager søren kragh
1             klæstrup gift i moské jeg er så lykkelig
2    teenager druknet efter skub i leg nu er kammer...
3         itfejl i europæisk flyveplansystem er rettet
4    voldtaget to timer på loftet over café  jeg vi...
Name: title, dtype: object

In [8]:
import nltk 
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop = stopwords.words('danish')
print(stop)


['og', 'i', 'jeg', 'det', 'at', 'en', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'af', 'for', 'ikke', 'der', 'var', 'mig', 'sig', 'men', 'et', 'har', 'om', 'vi', 'min', 'havde', 'ham', 'hun', 'nu', 'over', 'da', 'fra', 'du', 'ud', 'sin', 'dem', 'os', 'op', 'man', 'hans', 'hvor', 'eller', 'hvad', 'skal', 'selv', 'her', 'alle', 'vil', 'blev', 'kunne', 'ind', 'når', 'være', 'dog', 'noget', 'ville', 'jo', 'deres', 'efter', 'ned', 'skulle', 'denne', 'end', 'dette', 'mit', 'også', 'under', 'have', 'dig', 'anden', 'hende', 'mine', 'alt', 'meget', 'sit', 'sine', 'vor', 'mod', 'disse', 'hvis', 'din', 'nogle', 'hos', 'blive', 'mange', 'ad', 'bliver', 'hendes', 'været', 'thi', 'jer', 'sådan']


In [9]:
# Show amount of stop words
df['stopwords'] = df['title'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['title','stopwords']].head()

,title,stopwords
0,fem danskere i touren sunweb udtager søren kragh,1
1,klæstrup gift i moské jeg er så lykkelig,3
2,teenager druknet efter skub i leg nu er kammer...,4
3,itfejl i europæisk flyveplansystem er rettet,2
4,voldtaget to timer på loftet over café jeg vi...,8


In [10]:
# Remove stop words
df['title'] = df['title'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['title'].head()

0     fem danskere touren sunweb udtager søren kragh
1                    klæstrup gift moské så lykkelig
2          teenager druknet skub leg kammerat sigtet
3            itfejl europæisk flyveplansystem rettet
4    voldtaget to timer loftet café straffe fortjent
Name: title, dtype: object

In [11]:
# Get the 10 most common words
### Should this be all words above X count? Or something else?
freq = pd.Series(' '.join(df['title']).split()).value_counts()[:10]
freq

dansk      1380
kan         973
mand        854
så          781
ny          774
får         720
ved         618
år          612
danmark     599
danske      595
dtype: int64

In [12]:
# Remove the 10 most common words as they're not useful when classifying our data 
# (This allows us to catch some additional "stop words" not in our stop word library)
freq = list(freq.index)
df['title'] = df['title'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['title'].head()

0     fem danskere touren sunweb udtager søren kragh
1                       klæstrup gift moské lykkelig
2          teenager druknet skub leg kammerat sigtet
3            itfejl europæisk flyveplansystem rettet
4    voldtaget to timer loftet café straffe fortjent
Name: title, dtype: object

In [13]:
# Get the 10 least common words
### Should this be all words that are counted only once? Or something else?

freq = pd.Series(' '.join(df['title']).split()).value_counts()[-10:]
freq

pirates         1
middelmådigt    1
bullchef        1
udkig           1
vergara         1
drabssager      1
børnehøjde      1
ølproducent     1
mother          1
storholdet      1
dtype: int64

In [14]:
# Remove the rarest words
freq = list(freq.index)
df['title'] = df['title'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
df['title'].head()

0     fem danskere touren sunweb udtager søren kragh
1                       klæstrup gift moské lykkelig
2          teenager druknet skub leg kammerat sigtet
3            itfejl europæisk flyveplansystem rettet
4    voldtaget to timer loftet café straffe fortjent
Name: title, dtype: object

## Stemming

In [15]:
#Tokenize the title, e.g. put each word in its own array entry
from nltk.tokenize import word_tokenize
df['tokenized_title'] = df['title'].apply(word_tokenize) 
df.head()

,title,popularity,stopwords,tokenized_title
0,fem danskere touren sunweb udtager søren kragh,0,1,"[fem, danskere, touren, sunweb, udtager, søren..."
1,klæstrup gift moské lykkelig,1,3,"[klæstrup, gift, moské, lykkelig]"
2,teenager druknet skub leg kammerat sigtet,1,4,"[teenager, druknet, skub, leg, kammerat, sigtet]"
3,itfejl europæisk flyveplansystem rettet,0,2,"[itfejl, europæisk, flyveplansystem, rettet]"
4,voldtaget to timer loftet café straffe fortjent,1,8,"[voldtaget, to, timer, loftet, café, straffe, ..."


In [16]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('danish')
df["stemmed_title"] = df["tokenized_title"].apply(lambda x: [stemmer.stem(y) for y in x])
df.head()

,title,popularity,stopwords,tokenized_title,stemmed_title
0,fem danskere touren sunweb udtager søren kragh,0,1,"[fem, danskere, touren, sunweb, udtager, søren...","[fem, dansk, tour, sunweb, udtag, sør, kragh]"
1,klæstrup gift moské lykkelig,1,3,"[klæstrup, gift, moské, lykkelig]","[klæstrup, gift, moské, lyk]"
2,teenager druknet skub leg kammerat sigtet,1,4,"[teenager, druknet, skub, leg, kammerat, sigtet]","[teenag, drukn, skub, leg, kammerat, sigt]"
3,itfejl europæisk flyveplansystem rettet,0,2,"[itfejl, europæisk, flyveplansystem, rettet]","[itfejl, europæisk, flyveplansystem, ret]"
4,voldtaget to timer loftet café straffe fortjent,1,8,"[voldtaget, to, timer, loftet, café, straffe, ...","[voldtag, to, tim, loft, café, straf, fortjent]"


## Lemmatization

In [17]:
import lemmy
lemmatizer = lemmy.load("da")

In [18]:
lemmaWords = []
for i in df['title']:
    lemmatized_str = ""
    for word in str(i).split(' '):
        word = lemmatizer.lemmatize("", word)
        lemmatized_str = lemmatized_str+ " " + str(word[0])
    
    lemmaWords.append(lemmatized_str)
df['lemmatized_title'] = lemmaWords
df[['title','popularity','tokenized_title','stemmed_title','lemmatized_title']]

,title,popularity,tokenized_title,stemmed_title,lemmatized_title
0,fem danskere touren sunweb udtager søren kragh,0,"[fem, danskere, touren, sunweb, udtager, søren...","[fem, dansk, tour, sunweb, udtag, sør, kragh]",fem dansker tour sunweb udtage søren kragh
1,klæstrup gift moské lykkelig,1,"[klæstrup, gift, moské, lykkelig]","[klæstrup, gift, moské, lyk]",klæstrup gifte moské lykkelig
2,teenager druknet skub leg kammerat sigtet,1,"[teenager, druknet, skub, leg, kammerat, sigtet]","[teenag, drukn, skub, leg, kammerat, sigt]",teenager drukne skub leg kammerat sigte
3,itfejl europæisk flyveplansystem rettet,0,"[itfejl, europæisk, flyveplansystem, rettet]","[itfejl, europæisk, flyveplansystem, ret]",itfejl europæisk flyveplansystem rette
4,voldtaget to timer loftet café straffe fortjent,1,"[voldtaget, to, timer, loftet, café, straffe, ...","[voldtag, to, tim, loft, café, straf, fortjent]",voldtage to time loft café straffe fortjene
...,...,...,...,...,...
31513,helt hest politiet mangler borgernes hjælp nye...,0,"[helt, hest, politiet, mangler, borgernes, hjæ...","[helt, hest, politi, mangl, borg, hjælp, nye, ...",hel hest politi mangel borger hjælp ny transp...
31514,kæmpe ballade pokalfinale politiet brugte tåre...,1,"[kæmpe, ballade, pokalfinale, politiet, brugte...","[kæmp, ballad, pokalfinal, politi, brugt, tåre...",kæmpe ballade pokalfinale politi brugte tåreg...
31515,lavede sexbånd kardashian videre,0,"[lavede, sexbånd, kardashian, videre]","[laved, sexbånd, kardashian, vid]",lave sexbånde kardashian videre
31516,politiet bekræfter ronaldo afhøres,1,"[politiet, bekræfter, ronaldo, afhøres]","[politi, bekræft, ronaldo, afhør]",politi bekræfte ronaldo afhøre


# Testing Lemmatization

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.lemmatized_title, df.popularity, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23638,), (7880,), (23638,), (7880,))

In [20]:
# Use TF-IDF method
from sklearn.feature_extraction.text import TfidfVectorizer
vect_new = TfidfVectorizer(min_df=5).fit(X_train)
tfid_x_train = vect_new.transform(X_train)
tfid_x_test = vect_new.transform(X_test)

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

dummyC = DummyClassifier(strategy="most_frequent").fit(tfid_x_train, y_train)
print("Dummy Classifer")
print("Training set score: {:.3f}".format(dummyC.score(tfid_x_train, y_train)))
print("Test set score: {:.3f}".format(dummyC.score(tfid_x_test, y_test)))
print("-------------------------------------------")

logreg = LogisticRegression(max_iter=10000).fit(tfid_x_train, y_train)
print("Logistic Regression")
print("Training set score: {:.3f}".format(logreg.score(tfid_x_train, y_train)))
print("Test set score: {:.3f}".format(logreg.score(tfid_x_test, y_test)))
print("-------------------------------------------")

linearsvc = LinearSVC(max_iter=10000).fit(tfid_x_train, y_train)
print("Linear SVC")
print("Training set score: {:.3f}".format(linearsvc.score(tfid_x_train, y_train)))
print("Test set score: {:.3f}".format(linearsvc.score(tfid_x_test, y_test)))
print("-------------------------------------------")

dtc = DecisionTreeClassifier().fit(tfid_x_train, y_train)
print("Decision Tree")
print("Training set score: {:.3f}".format(dtc.score(tfid_x_train, y_train)))
print("Test set score: {:.3f}".format(dtc.score(tfid_x_test, y_test)))
print("-------------------------------------------")

rfc = RandomForestClassifier().fit(tfid_x_train, y_train)
print("Random Forest")
print("Training set score: {:.3f}".format(rfc.score(tfid_x_train, y_train)))
print("Test set score: {:.3f}".format(rfc.score(tfid_x_test, y_test)))
print("-------------------------------------------")

Dummy Classifer
Training set score: 0.502
Test set score: 0.498
-------------------------------------------
Logistic Regression
Training set score: 0.757
Test set score: 0.662
-------------------------------------------
Linear SVC
Training set score: 0.771
Test set score: 0.657
-------------------------------------------
Decision Tree
Training set score: 0.990
Test set score: 0.610
-------------------------------------------
Random Forest
Training set score: 0.990
Test set score: 0.647
-------------------------------------------
